In [12]:
import pandas as pd
df = pd.read_csv("dataset_balanced.csv")
df["labels"].sum()
df["labels"].shape
positive = df[df["labels"]==1].sample(n=500)
negative = df[df["labels"]==0].sample(n=500)
df2 = pd.concat([positive,negative])
df2.to_csv("dataset_final.csv")

In [14]:
!pip install transformers
!pip install datasets
!pip install torchmetrics
!pip install gradio

import gradio
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
import torchmetrics
from torchmetrics.classification.f_beta import F1Score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
# import the dataset into a dataset class
dataset = load_dataset("csv", data_files="dataset_final.csv", split="train")

# summarize dataset
size = len(dataset["text"])
print(f"Found {size} examples to train on")
print("Schema:")
print(dataset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-7c76c83deac66d0f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.
Found 1000 examples to train on
Schema:
Dataset({
    features: ['Unnamed: 0', 'text', 'labels'],
    num_rows: 1000
})


In [16]:

split_dataset = dataset.train_test_split(test_size=0.2, shuffle=True)

train_ds = split_dataset["train"]
test_ds = split_dataset["test"]

max_length = 100

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-cased")

def tokenize_function(examples):
  return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=max_length)

train_ds = train_ds.map(tokenize_function)
test_ds = test_ds.map(tokenize_function)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/9d7568e4b20ed5db15ee30e99c7219bde9990762/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/9d7568e4b20ed5db15ee30e99c7219bde9990762/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/9d7568e4b20ed5db15ee30e99c7219bde9990762/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_

  0%|          | 0/800 [00:00<?, ?ex/s]

  0%|          | 0/200 [00:00<?, ?ex/s]

In [17]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-cased",
                                                            num_labels=2)

# set up the training args (where to keep checkpoints + when to evaluate)
training_args = TrainingArguments(output_dir="test_trainer", 
                                  evaluation_strategy="steps",
                                  eval_steps=100,
                                  num_train_epochs=30,
                                  logging_steps=100,
                                  learning_rate=5e-7)

#
#define metrics to compute on each epoch
f1 = torchmetrics.classification.BinaryF1Score()
acc = torchmetrics.classification.BinaryAccuracy()
def compute_metrics(eval_pred):
  f1.reset()
  acc.reset()
  logits, labels = eval_pred
  logits = torch.tensor(logits)
  labels = torch.tensor(labels)
  acc(logits, F.one_hot(labels, num_classes=2))
  f1(logits, F.one_hot(labels, num_classes=2))
  return {"Accuracy": acc.compute(), "F1 score": f1.compute() }

# trainer class that handles the model training loops
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/9d7568e4b20ed5db15ee30e99c7219bde9990762/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 28996
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/9d7568e4b20ed5db15ee30e99c7219bde9990762/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_pro

In [18]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, Unnamed: 0. If text, Unnamed: 0 are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 800
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3000
  Number of trainable parameters = 65783042


Step,Training Loss,Validation Loss,Accuracy,F1 score
100,0.696700,0.689495,0.525000,0.527363
200,0.691100,0.686991,0.540000,0.544554
300,0.689100,0.684126,0.580000,0.580000
400,0.683800,0.680472,0.660000,0.663366
500,0.682500,0.675939,0.747500,0.760095
600,0.678200,0.669953,0.777500,0.792541
700,0.671200,0.662282,0.790000,0.804651
800,0.662100,0.653041,0.792500,0.807425
900,0.654300,0.641558,0.792500,0.806527
1000,0.640700,0.628317,0.797500,0.807601


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, Unnamed: 0. If text, Unnamed: 0 are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, Unnamed: 0. If text, Unnamed: 0 are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, Unnamed: 0. If text, Unnamed: 0 are not expected by `DistilBertForSequenceClassification.forward`,  you can safely i

TrainOutput(global_step=3000, training_loss=0.5635591697692871, metrics={'train_runtime': 306.511, 'train_samples_per_second': 78.301, 'train_steps_per_second': 9.788, 'total_flos': 620940931200000.0, 'train_loss': 0.5635591697692871, 'epoch': 30.0})

In [8]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-cased")
max_length = 512

def predict(inputs):
  tokenized_input = tokenizer(inputs,
                              padding='max_length',
                              truncation=True,
                              max_length=max_length, return_tensors="pt")
  #print(tokenized_input)
  preds = model(input_ids=tokenized_input["input_ids"].to("cuda"), attention_mask=tokenized_input["attention_mask"].to("cuda"))
  pred = torch.argmax(preds["logits"], dim=-1)
  print(preds["logits"])
  return "Conspiracy" if pred == 1 else "Not Conspiracy"
  print(preds)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/9d7568e4b20ed5db15ee30e99c7219bde9990762/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/9d7568e4b20ed5db15ee30e99c7219bde9990762/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/9d7568e4b20ed5db15ee30e99c7219bde9990762/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_

In [29]:
predict("Theorists also believe that the US government staged the moon landing to win the Cold War against the Soviet Union.")

tensor([[-0.1666,  0.3343]], device='cuda:0', grad_fn=<AddmmBackward0>)


'Conspiracy'

In [22]:
import gradio as gr

demo = gr.Interface(fn=predict, inputs="text", outputs="text")

demo.launch()   

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`

Using Embedded Colab Mode (NEW). If you have issues, please use share=True and file an issue at https://github.com/gradio-app/gradio/
Note: opening the browser inspector may crash Embedded Colab Mode.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

(<gradio.routes.App at 0x7f3b1bf64a10>, 'http://127.0.0.1:7860/', None)